# マイクロマウスのターン生成

## 概要
マイクロマウス競技において、高速な走行を行うためにはスラロームと呼ばれる車体を止めずにターンをする動作が必要とされます。
このプログラムでは、スラローム走行に必要な軌跡を生成します。
    

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt 
import matplotlib.ticker as tick
import numpy as np 
from numba import jit
import math 
from itertools import product

from ipywidgets import interact, interactive, fixed


In [2]:
# molifireの積分値を計算するための関数


#y = math.e / a * np.exp( (-1)/(1- np.abs(2*x/b -1)**c))

def calc_rk4_delta(f, t, y, dt):
    k1 = f(t, y)
    k2 = f(t+dt*0.5, t+dt*0.5*k1)
    k3 = f(t+dt*0.5, t+dt*0.5*k2)
    k4 = f(t+dt,t+dt*k3)
    return (k1+2*k2+2*k3+k4)* self.delta_t /6.0

@jit('f8(f8, f8, f8, f8)')
def moli(x,a,b,c):
    if c <= 0.0:
        return 0.0    
    if x <= 0.0:
        return 0.0
    elif x >= b:        
        return 0.0
    else:
        return math.e / a * math.exp( (-1)/(1- abs(2*x/b -1)**c))

@jit('f8(f8)')
def int_moli(c):
    delta_t = 0.000001
    x_list = np.arange(0.0,1.0,delta_t)

    S = 0.0
    
    # delta_tを0.000001まで細かくとったら以下の2つのfor文の差はfloatで表現できる最小誤差以下だった
    for x in x_list:        
        k1 = moli(x, 1.0, 1.0, c)
        k2 = moli(x+delta_t*0.5, 1.0, 1.0, c)
        k3 = k2
        k4 = moli(x+delta_t,1.0, 1.0, c)
        delta_y = delta_t/6.0 * (k1 + 2.0 * k2+ 2.0*k3 + k4)
        S = S + delta_y        
    #for x in x_list: 
    #    y = moli(x, 1.0, 1.0, c)
    #    S = S + delta_t * y
    return S

In [3]:
#for i in np.arange(0.0, 6.01, 0.01):    
#    print("{:.9f}".format(int_moli(i))+"f,")

In [4]:

# 軌跡を作るためのクラス
class CalcTrajWithBeta:
    shape_factor = 0
    path_length = 0
    target_ang = 0
    delta_t = 0
    tread = 0
    start_ang = 0
    start_x = 0
    start_y = 0
    v = 0
    cp = 0    
    
    t = None
    omega = None
    alpha = None
    ang   = None    
    c_traj_x = None
    c_traj_y = None
    l_traj_x = None
    l_traj_y = None
    r_traj_x = None
    r_traj_y = None

    t_beta = None
    omega_beta = None
    alpha_beta = None
    ang_beta   = None        
    beta  = None    
    
    c_traj_x_beta = None
    c_traj_y_beta = None
    l_traj_x_beta = None
    l_traj_y_beta = None
    r_traj_x_beta = None
    r_traj_y_beta = None

    
    
    # 軌跡のパラメータをセットする
    def set_param(self, shape_factor_, path_length_, target_ang_, delta_t_, tread_, v_, cp_,start_x_=0, start_y_=0, start_ang_=0):
        self.shape_factor = shape_factor_
        self.path_length = path_length_
        self.target_ang = target_ang_
        self.delta_t = delta_t_
        self.tread = tread_
        self.start_x = start_x_
        self.start_y = start_y_
        self.start_ang = start_ang_
        self.v= v_
        self.cp = cp_
        
    # 軌跡を計算する
        
    def calc_traj(self):        
        path_end_time = self.path_length/ self.v
        self.t = np.arange(0.0, path_end_time, self.delta_t)
        
        # a * b * S = target_ang
        c = self.shape_factor
        b = self.path_length
        S = int_moli(c)
        a = 1/(self.target_ang/b/S)
                
        # 角速度
        self.omega = np.zeros(len(self.t),dtype=np.float64)
        for i in range(len(self.t)):
            self.omega[i] = self.v * moli(self.t[i]*self.v, a, b, c)                
        
        # 角速度と時間のリストから角速度が限りなく0に近い部分をトリミング
        index = 0
        del_count = 0
        while index < len(self.omega):
            if self.omega[index] < 0.01745: # 0.01745 rad/s = 1deg/s
                self.omega = np.delete(self.omega, index)
                self.t     = np.delete(self.t, index)
                path_end_time = path_end_time - self.delta_t
                index = 0
                del_count = del_count + 1                
            else:
                index = index + 1
        #print ("del_count", del_count)
        
        # 角加速度
        self.alpha = np.gradient(self.omega) / self.delta_t
        
        # 角度
        self.ang = np.zeros(len(self.omega))
        self.ang[0] = self.start_ang;
        for i in range(len(self.t)):
            if i !=0:
                k1 = self.v * moli(self.t[i]*self.v, a, b, c)
                k2 = self.v * moli( (self.t[i] + self.delta_t/2)*self.v, a, b, c)
                k3 = k2
                k4 = self.v * moli( (self.t[i] + self.delta_t)*self.v, a, b, c)
                
                self.ang[i] = self.ang[i-1] + self.delta_t/6.0 * (k1 + 2.0*k2 + 2.0*k3 + k4)
        #for i,ele in enumerate(self.omega):
        #    if i != 0:
        #        self.ang[i] = self.ang[i-1] + self.delta_t * ele 
                
        # スリップ角
        self.t_beta = self.t.copy()
        self.omega_beta = self.omega.copy()
        self.alpha_beta = self.alpha.copy()
        self.ang_beta   = self.ang.copy()
        self.beta = np.zeros(len(self.omega))
        
        def f(t,y):
            return -self.cp * y/(self.v * math.cos(y)) - self.v * moli(t*self.v, a, b, c)
                
        i = 0
        slip_after_count = 0
        while i < len(self.omega_beta):
            if i != 0:
                k1 = f(self.t_beta[i-1], self.beta[i-1])
                k2 = f(self.t_beta[i-1]+self.delta_t*0.5, self.beta[i-1]+self.delta_t*0.5*k1)
                k3 = f(self.t_beta[i-1]+self.delta_t*0.5, self.beta[i-1]+self.delta_t*0.5*k2)
                k4 = f(self.t_beta[i-1]+self.delta_t, self.beta[i-1]+self.delta_t*k3)
                               
                self.beta[i] = self.beta[i-1] + (k1+2*k2+2*k3+k4)* self.delta_t /6.0                        

            if i == len(self.omega_beta)-1 and abs(self.beta[i]) > 0.01745:
                slip_after_count = slip_after_count + 1
                self.t_beta = np.append(self.t_beta, self.t_beta[-1]+self.delta_t)
                self.omega_beta = np.append(self.omega_beta, 0.0)
                self.alpha_beta = np.append(self.alpha_beta, 0.0)
                self.ang_beta = np.append(self.ang_beta, self.ang_beta[-1])
                self.beta = np.append(self.beta, 0.0)                              
            i = i + 1
        #print("slip_after_count:", slip_after_count)
        
        # スリップ角考慮なしの軌跡を求める       
        self.c_traj_x = np.zeros(len(self.omega))
        self.c_traj_y = np.zeros(len(self.omega))        
        self.l_traj_x = np.zeros(len(self.omega))
        self.l_traj_y = np.zeros(len(self.omega))
        self.r_traj_x = np.zeros(len(self.omega))
        self.r_traj_y = np.zeros(len(self.omega))
        self.c_traj_x[0] = self.start_x
        self.c_traj_y[0] = self.start_y        
        self.r_traj_x[0] = self.c_traj_x[0] + math.sin(self.start_ang)*self.tread/2
        self.r_traj_y[0] = self.c_traj_y[0] - math.cos(self.start_ang)*self.tread/2
        self.l_traj_x[0] = self.c_traj_x[0] - math.sin(self.start_ang)*self.tread/2
        self.l_traj_y[0] = self.c_traj_y[0] + math.cos(self.start_ang)*self.tread/2
        
        for i,ele in enumerate(self.ang):
            if i != 0:
                
                if self.omega[i] > 0.087: # 0.087rad = 5deg
                    self.c_traj_x[i] = self.c_traj_x[i-1] + self.v * math.cos(ele) * math.sin(self.omega[i]* self.delta_t * 0.5) / (self.omega[i] * 0.5)
                    self.c_traj_y[i] = self.c_traj_y[i-1] + self.v * math.sin(ele) * math.sin(self.omega[i]* self.delta_t * 0.5) / (self.omega[i] * 0.5)
                else:
                    self.c_traj_x[i] = self.c_traj_x[i-1] + self.v * math.cos(ele) * self.delta_t
                    self.c_traj_y[i] = self.c_traj_y[i-1] + self.v * math.sin(ele) * self.delta_t
                                    
                self.r_traj_x[i] = self.c_traj_x[i] + math.sin(ele)*self.tread/2
                self.r_traj_y[i] = self.c_traj_y[i] - math.cos(ele)*self.tread/2

                self.l_traj_x[i] = self.c_traj_x[i] - math.sin(ele)*self.tread/2
                self.l_traj_y[i] = self.c_traj_y[i] + math.cos(ele)*self.tread/2

        # スリップ角考慮ありの軌跡を求める
        self.c_traj_x_beta = np.zeros(len(self.omega_beta))
        self.c_traj_y_beta = np.zeros(len(self.omega_beta))        
        self.l_traj_x_beta = np.zeros(len(self.omega_beta))
        self.l_traj_y_beta = np.zeros(len(self.omega_beta))
        self.r_traj_x_beta = np.zeros(len(self.omega_beta))
        self.r_traj_y_beta = np.zeros(len(self.omega_beta))
        self.c_traj_x_beta[0] = self.start_x
        self.c_traj_y_beta[0] = self.start_y        
        self.r_traj_x_beta[0] = self.c_traj_x_beta[0] + math.sin(self.start_ang)*self.tread/2
        self.r_traj_y_beta[0] = self.c_traj_y_beta[0] - math.cos(self.start_ang)*self.tread/2
        self.l_traj_x_beta[0] = self.c_traj_x_beta[0] - math.sin(self.start_ang)*self.tread/2
        self.l_traj_y_beta[0] = self.c_traj_y_beta[0] + math.cos(self.start_ang)*self.tread/2
                
        for i,ele in enumerate(self.ang_beta):
            if i != 0:                
                # スリップ角考慮あり曲線
                if self.omega_beta[i] > 0.087: # 0.087rad = 5deg
                    self.c_traj_x_beta[i] = self.c_traj_x_beta[i-1] + self.v * math.cos(ele + self.beta[i]) * math.sin(self.omega[i]* self.delta_t * 0.5) / (self.omega[i] * 0.5)
                    self.c_traj_y_beta[i] = self.c_traj_y_beta[i-1] + self.v * math.sin(ele + self.beta[i]) * math.sin(self.omega[i]* self.delta_t * 0.5) / (self.omega[i] * 0.5)
                else:
                    self.c_traj_x_beta[i] = self.c_traj_x_beta[i-1] + self.v * math.cos(ele + self.beta[i]) * self.delta_t
                    self.c_traj_y_beta[i] = self.c_traj_y_beta[i-1] + self.v * math.sin(ele + self.beta[i]) * self.delta_t

                self.r_traj_x_beta[i] = self.c_traj_x_beta[i] + math.sin(ele)*self.tread/2
                self.r_traj_y_beta[i] = self.c_traj_y_beta[i] - math.cos(ele)*self.tread/2

                self.l_traj_x_beta[i] = self.c_traj_x_beta[i] - math.sin(ele)*self.tread/2
                self.l_traj_y_beta[i] = self.c_traj_y_beta[i] + math.cos(ele)*self.tread/2                
            
    # 軌跡が終わったポイントのx座標
    def get_end_x(self):
        return self.c_traj_x[len(self.c_traj_x)-1]
    # 軌跡が終わったポイントのy座標    
    def get_end_y(self):
        return self.c_traj_y[len(self.c_traj_y)-1]
    
    # x方向の移動距離
    def get_move_x(self):
        return self.c_traj_x[len(self.c_traj_x)-1] - self.start_x
    
    # y方向の移動距離
    def get_move_y(self):
        return self.c_traj_y[len(self.c_traj_y)-1] - self.start_y
        
    # 速度一定で走ったときの時間と角速度のリストを返す
    def get_omega(self):
        return self.t, self.omega
    
    # 速度一定で走ったときの時間と角速度(degree)のリストを返す
    def get_omega_degree(self):
        t, rad = self.get_omega() 
        return t, rad * 180 /math.pi
    
    # 速度一定で走ったときの時間と角加速度のリストを返す
    def get_alpha(self):
        return self.t,self.alpha 

    # 速度一定で走ったときの時間と角加速度(degree)のリストを返す
    def get_alpha_degree(self):
        t, alpha = self.get_alpha()
        return t, alpha * 180/math.pi

    # 速度一定で走ったときの時間と角度のリストを返す
    def get_ang(self):        
        return self.t, self.ang    
    
    # 速度一定で走ったときの時間と角度(degree)のリストを返す
    def get_ang_degree(self):
        t, ang = self.get_ang()
        return t, ang * 180/math.pi
        
    # 速度一定で走ったときの時間と向進加速度のリストを返す
    def get_cent_acc(self):
        return self.t, self.v * self.omega 

    # 速度一定で走ったときの時間とスリップ角のリストを返す
    def get_beta(self):
        return self.t_beta, self.beta
    
    # 速度一定で走ったときの時間とスリップ角(degree)のリストを返す
    def get_beta_degree(self):
        t, rad = self.get_beta() 
        return t, rad * 180 /math.pi

    # スリップ角有の軌跡が終わったポイントのx座標
    def get_end_x_beta(self):
        return self.c_traj_x_beta[-1]
    
    # スリップ角有の軌跡が終わったポイントのy座標    
    def get_end_y_beta(self):
        return self.c_traj_y_beta[-1]
    
    # スリップ角有のx方向の移動距離
    def get_move_x_beta(self):
        return self.c_traj_x_beta[-1] - self.start_x
    
    # スリップ角有のy方向の移動距離
    def get_move_y_beta(self):
        return self.c_traj_y_beta[-1] - self.start_y
        

In [5]:
def simulate(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length, v):
    traj_curve = CalcTrajWithBeta()
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, v_=v, cp_ = cp, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()

    # 前後距離の算出
    X = end_x - start_x - traj_curve.get_move_x_beta()
    Y = end_y - start_y - traj_curve.get_move_y_beta()
    end_ang = start_ang + target_ang
    sin_start_ang = math.sin(start_ang * math.pi/180.0)
    cos_start_ang = math.cos(start_ang * math.pi/180.0)
    sin_end_ang = math.sin(end_ang * math.pi/180.0)
    cos_end_ang = math.cos(end_ang * math.pi/180.0)
        
    d_pre_beta = (sin_end_ang   * X - cos_end_ang   * Y) / (cos_start_ang * sin_end_ang   - cos_end_ang   * sin_start_ang)
    d_fol_beta = (sin_start_ang * X - cos_start_ang * Y) / (cos_end_ang   * sin_start_ang - cos_start_ang * sin_end_ang)
    
    # 値格納用変数の初期化
    t, omega = traj_curve.get_omega_degree()
    t, alpha = traj_curve.get_alpha_degree()
    t, ang   = traj_curve.get_ang_degree()
    t, cent_acc = traj_curve.get_cent_acc()
    t_beta, beta  = traj_curve.get_beta_degree()    
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([traj_curve.r_traj_x_beta[0],traj_curve.r_traj_x_beta[0] + d_pre_beta*cos_start_ang], \
             [traj_curve.r_traj_y_beta[0],traj_curve.r_traj_y_beta[0] + d_pre_beta*sin_start_ang], color="blue") 
    
    plt.plot([start_x, traj_curve.c_traj_x_beta[0] + d_pre_beta * cos_start_ang], \
             [start_y, traj_curve.c_traj_y_beta[0] + d_pre_beta * sin_start_ang], color="blue") 
    
    plt.plot([traj_curve.l_traj_x_beta[0], traj_curve.l_traj_x_beta[0] + d_pre_beta*cos_start_ang], \
             [traj_curve.l_traj_y_beta[0], traj_curve.l_traj_y_beta[0] + d_pre_beta*sin_start_ang], color="blue") 
        
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.r_traj_x_beta + d_pre_beta*cos_start_ang, \
             traj_curve.r_traj_y_beta + d_pre_beta*sin_start_ang, color="orange")
    
    plt.plot(traj_curve.c_traj_x_beta + d_pre_beta*cos_start_ang, 
             traj_curve.c_traj_y_beta + d_pre_beta*sin_start_ang, color="orange")
    
    plt.plot(traj_curve.l_traj_x_beta + d_pre_beta*cos_start_ang, \
             traj_curve.l_traj_y_beta + d_pre_beta*sin_start_ang, color="orange")
        
    # 直後部分の軌跡描画
    sin_end_p90_ang = math.sin((end_ang+90) * math.pi/180.0)
    cos_end_p90_ang = math.cos((end_ang+90) * math.pi/180.0)
    sin_end_m90_ang = math.sin((end_ang-90) * math.pi/180.0)
    cos_end_m90_ang = math.cos((end_ang-90) * math.pi/180.0)
    
    plt.plot([traj_curve.get_end_x_beta() + d_pre_beta*cos_start_ang + tread/2*cos_end_p90_ang, end_x+tread/2*cos_end_p90_ang], \
             [traj_curve.get_end_y_beta() + d_pre_beta*sin_start_ang + tread/2*sin_end_p90_ang, end_y+tread/2*sin_end_p90_ang], color="blue") 

    plt.plot([traj_curve.get_end_x_beta() + d_pre_beta*cos_start_ang, end_x], \
             [traj_curve.get_end_y_beta() + d_pre_beta*sin_start_ang, end_y], color="blue")

    plt.plot([traj_curve.get_end_x_beta() + d_pre_beta*cos_start_ang + tread/2*cos_end_m90_ang, end_x+tread/2*cos_end_m90_ang], \
             [traj_curve.get_end_y_beta() + d_pre_beta*sin_start_ang + tread/2*sin_end_m90_ang, end_y+tread/2*sin_end_m90_ang], color="blue")
                        
    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.title("trajectory")
    plt.axis([-0.18, 0.18, -0.18, 0.18])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.09+ 0.045-0.003, j*0.09+0.045-0.003], width=0.006, height=0.006) )
    plt.show()

      
    #print("d_pre x[m] " + str(d_pre))
    #print("d_fol y[m] " + str(d_fol))
    #print("total length[m] " + str(d_pre+d_fol+path_length__) )
    print("d_pre_beta[m] " + str(d_pre_beta))
    print("d_fol_beta[m] " + str(d_fol_beta))
    print("total length[m] " + str(d_pre_beta+d_fol_beta+traj_curve.t_beta[-1] * v) )

    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.xlim(t_beta[0], t_beta[-1])
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.title("beta [deg] ")
    plt.grid()
    plt.xlim(t_beta[0], t_beta[-1])
    plt.plot(t_beta,-beta, label="beta", color="red")
        
    plt.subplot(4, 1, 3)
    plt.plot(t,alpha)
    plt.xlim(t_beta[0], t_beta[-1])
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.xlim(t_beta[0], t_beta[-1])
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()


def simulate_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length, v):
    traj_curve = CalcTrajWithBeta()
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, v_=v, cp_ = cp, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()

    # 前後距離の算出
    X = end_x - start_x - traj_curve.get_move_x_beta()
    Y = end_y - start_y - traj_curve.get_move_y_beta()
    end_ang = start_ang + target_ang
    sin_start_ang = math.sin(start_ang * math.pi/180.0)
    cos_start_ang = math.cos(start_ang * math.pi/180.0)
    sin_end_ang = math.sin(end_ang * math.pi/180.0)
    cos_end_ang = math.cos(end_ang * math.pi/180.0)
        
    d_pre_beta = (sin_end_ang   * X - cos_end_ang   * Y) / (cos_start_ang * sin_end_ang   - cos_end_ang   * sin_start_ang)
    d_fol_beta = (sin_start_ang * X - cos_start_ang * Y) / (cos_end_ang   * sin_start_ang - cos_start_ang * sin_end_ang)
    
    # 値格納用変数の初期化
    t, omega = traj_curve.get_omega_degree()
    t, alpha = traj_curve.get_alpha_degree()
    t, ang   = traj_curve.get_ang_degree()
    t, cent_acc = traj_curve.get_cent_acc()
    t_beta, beta  = traj_curve.get_beta_degree()    
    
    return (d_pre_beta, d_fol_beta, np.max(np.abs(beta)), np.max(t_beta))

In [6]:
def simulate180(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length_min, path_length_max, v):
    traj_curve = CalcTrajWithBeta()    
    error_y = 1.0
    
    # 最小化するターゲット関数
    def calc_y_error(pl_):
        traj_curve = CalcTrajWithBeta()
        traj_curve.set_param(shape_factor_=shape_factor, path_length_= pl_, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, v_=v, cp_ = cp, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)

        traj_curve.calc_traj()
        
        error_y_ = traj_curve.get_move_y_beta() - end_y
        return error_y_, traj_curve
    
    while abs(error_y) > 0.0001:
        pl = 0.5 * (path_length_min + path_length_max)
        error_y, traj_curve = calc_y_error(pl)
        path_length = pl
        if(error_y > 0):
            path_length_max = pl
        else:
            path_length_min = pl
        print("error_y: " + str(error_y))
        print("path_length: " + str(pl))
        print("min, max :",path_length_min, path_length_max)
 
    # 前後距離の算出
    X = end_x - start_x - traj_curve.get_move_x_beta()
    Y = end_y - start_y - traj_curve.get_move_y_beta()
    end_ang = start_ang + target_ang
    sin_start_ang = math.sin(start_ang * math.pi/180.0)
    cos_start_ang = math.cos(start_ang * math.pi/180.0)
    sin_end_ang = math.sin(end_ang * math.pi/180.0)
    cos_end_ang = math.cos(end_ang * math.pi/180.0)
        
    #d_pre_beta = (sin_end_ang   * X - cos_end_ang   * Y) / (cos_start_ang * sin_end_ang   - cos_end_ang   * sin_start_ang)
    #d_fol_beta = (sin_start_ang * X - cos_start_ang * Y) / (cos_end_ang   * sin_start_ang - cos_start_ang * sin_end_ang)
    d_pre_beta =0.027
    d_fol_beta = d_pre_beta + traj_curve.get_move_x_beta()     
    
    # 値格納用変数の初期化
    t, omega = traj_curve.get_omega_degree()
    t, alpha = traj_curve.get_alpha_degree()
    t, ang   = traj_curve.get_ang_degree()
    t, cent_acc = traj_curve.get_cent_acc()
    t_beta, beta  = traj_curve.get_beta_degree()    
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre_beta],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre_beta],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre_beta],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 
    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.c_traj_x_beta + d_pre_beta, traj_curve.c_traj_y_beta, color="blue")
    plt.plot(traj_curve.r_traj_x_beta + d_pre_beta, traj_curve.r_traj_y_beta, color="blue")
    plt.plot(traj_curve.l_traj_x_beta + d_pre_beta, traj_curve.l_traj_y_beta, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.get_end_x_beta()+ d_pre_beta , end_x],[traj_curve.get_end_y_beta()+tread/2, traj_curve.get_end_y_beta()+tread/2]  ,color="red") 
    plt.plot([traj_curve.get_end_x_beta()+ d_pre_beta , end_x],[traj_curve.get_end_y_beta(), traj_curve.get_end_y_beta()]  ,color="red") 
    plt.plot([traj_curve.get_end_x_beta()+ d_pre_beta , end_x],[traj_curve.get_end_y_beta()-tread/2, traj_curve.get_end_y_beta()-tread/2]  ,color="red") 
               
    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.title("trajectory")
    plt.axis([-0.18, 0.18, -0.18, 0.18])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.09+ 0.045-0.003, j*0.09+0.045-0.003], width=0.006, height=0.006) )
    plt.show()

      
    #print("d_pre x[m] " + str(d_pre))
    #print("d_fol y[m] " + str(d_fol))
    #print("total length[m] " + str(d_pre+d_fol+path_length__) )
    print("d_pre_beta x[m] " + str(d_pre_beta))
    print("d_fol_beta y[m] " + str(d_fol_beta))
    print("total length[m] " + str(d_pre_beta+d_fol_beta+traj_curve.t_beta[-1] * v) )

    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.xlim(t_beta[0], t_beta[-1])
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.title("beta [deg] ")
    plt.grid()
    plt.xlim(t_beta[0], t_beta[-1])
    plt.plot(t_beta,-beta, label="beta", color="red")
        
    plt.subplot(4, 1, 3)
    plt.plot(t,alpha)
    plt.xlim(t_beta[0], t_beta[-1])
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.xlim(t_beta[0], t_beta[-1])
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()

def simulate180_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length_min, path_length_max, v):
    traj_curve = CalcTrajWithBeta()    
    error_y = 1.0
    
    # 最小化するターゲット関数
    def calc_y_error(pl_):
        traj_curve = CalcTrajWithBeta()
        traj_curve.set_param(shape_factor_=shape_factor, path_length_= pl_, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, v_=v, cp_ = cp, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)

        traj_curve.calc_traj()
        
        error_y_ = traj_curve.get_move_y_beta() - end_y
        return error_y_, traj_curve
    
    while abs(error_y) > 0.0001:
        pl = 0.5 * (path_length_min + path_length_max)
        error_y, traj_curve = calc_y_error(pl)
        path_length = pl
        if(error_y > 0):
            path_length_max = pl
        else:
            path_length_min = pl
 
    # 前後距離の算出
    X = end_x - start_x - traj_curve.get_move_x_beta()
    Y = end_y - start_y - traj_curve.get_move_y_beta()
    end_ang = start_ang + target_ang
    sin_start_ang = math.sin(start_ang * math.pi/180.0)
    cos_start_ang = math.cos(start_ang * math.pi/180.0)
    sin_end_ang = math.sin(end_ang * math.pi/180.0)
    cos_end_ang = math.cos(end_ang * math.pi/180.0)
        
    #d_pre_beta = (sin_end_ang   * X - cos_end_ang   * Y) / (cos_start_ang * sin_end_ang   - cos_end_ang   * sin_start_ang)
    #d_fol_beta = (sin_start_ang * X - cos_start_ang * Y) / (cos_end_ang   * sin_start_ang - cos_start_ang * sin_end_ang)
    d_pre_beta =0.027
    d_fol_beta = d_pre_beta + traj_curve.get_move_x_beta()     
    
    # 値格納用変数の初期化
    t, omega = traj_curve.get_omega_degree()
    t, alpha = traj_curve.get_alpha_degree()
    t, ang   = traj_curve.get_ang_degree()
    t, cent_acc = traj_curve.get_cent_acc()
    t_beta, beta  = traj_curve.get_beta_degree()        
    
    return (d_pre_beta, d_fol_beta, np.max(np.abs(beta)), np.max(t_beta))

## ハーフ90度ターン


In [7]:
from pylab import rcParams
rcParams['figure.figsize'] = 8,8

# 設定パラメータ
tread = 0.038
delta_t = 0.001
start_x = -0.045
start_y = 0.00
end_x = 0.0
end_y = 0.045
target_ang = 90
start_ang = 0.0
cp = 100

shape_factor__ = 4
path_length__ = 0.05
v__ = 0.3

print(" v,     pre,      fol,      beta_max,  end_time")
for v in np.arange(0.1, 1.6, 0.1):
    _pre, _fol, _beta_max, _end_time = simulate_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor__, path_length__, v)
    print('{:>5.2f}'.format(v), '{:>9.5f}'.format(_pre), '{:>9.5f}'.format(_fol), '{:>7.2f}'.format(_beta_max), '{:>10.3f}'.format(_end_time) )



@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.005, 0.36, 0.005),  v = (0.01, 1.0, 0.1)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    simulate(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length, v)

 v,     pre,      fol,      beta_max,  end_time
 0.10   0.01569   0.01559    0.24      0.489
 0.20   0.01544   0.01564    0.94      0.245
 0.30   0.01497   0.01607    2.12      0.163
 0.40   0.01468   0.01668    3.76      0.122
 0.50   0.01367   0.01563    5.86      0.101
 0.60   0.01281   0.01352    8.38      0.089
 0.70   0.01168   0.01043   11.25      0.082
 0.80   0.00960   0.00634   14.39      0.078
 0.90   0.00792   0.00137   17.69      0.076
 1.00   0.00601  -0.00516   21.04      0.076
 1.10   0.00390  -0.01165   24.36      0.076
 1.20   0.00156  -0.01930   27.59      0.077
 1.30  -0.00102  -0.02841   30.69      0.079
 1.40  -0.00381  -0.03788   33.61      0.081
 1.50  -0.00683  -0.04770   36.38      0.083


interactive(children=(FloatSlider(value=4.0, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

## ハーフ大周り90度

In [8]:
from pylab import rcParams
rcParams['figure.figsize'] = 8,8

# 設定パラメータ
tread = 0.038
delta_t = 0.001
start_x = -0.09
start_y = 0.00
end_x = 0.0
end_y = 0.09
target_ang = 90
start_ang = 0.0
cp = 250

shape_factor__ = 2
path_length__ = 0.11
v__ = 1.5

print(" v,     pre,      fol,      beta_max,  end_time")
for v in np.arange(1.0, 2.5, 0.1):
    _pre, _fol, _beta_max, _end_time = simulate_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor__, path_length__, v)
    print('{:>5.2f}'.format(v), '{:>9.5f}'.format(_pre), '{:>9.5f}'.format(_fol), '{:>7.2f}'.format(_beta_max), '{:>10.3f}'.format(_end_time) )



@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 2.0, 0.1)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    simulate(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length, v)


 v,     pre,      fol,      beta_max,  end_time
 1.00   0.02602   0.03097    5.37      0.106
 1.10   0.02567   0.03196    6.47      0.096
 1.20   0.02404   0.03138    7.65      0.089
 1.30   0.02338   0.02966    8.92      0.084
 1.40   0.02261   0.02619   10.25      0.081
 1.50   0.02175   0.02335   11.64      0.078
 1.60   0.02080   0.02114   13.07      0.075
 1.70   0.01811   0.01781   14.55      0.073
 1.80   0.01687   0.01314   16.05      0.072
 1.90   0.01554   0.00870   17.56      0.071
 2.00   0.01413   0.00447   19.08      0.070
 2.10   0.01261  -0.00163   20.59      0.070
 2.20   0.01105  -0.00552   22.10      0.069
 2.30   0.00936  -0.01150   23.59      0.069
 2.40   0.00756  -0.01986   25.06      0.070


interactive(children=(FloatSlider(value=2.0, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

## ハーフ180度ターン
180度ターンは曲線長と角度を指定するだけでうまく終点と始点が結べないため注意。
カーブの終点が180度ターンの終了点になるように曲線長を2分探索で求めている。

In [9]:
from pylab import rcParams
rcParams['figure.figsize'] = 8,8

# 設定パラメータ
tread = 0.038
delta_t = 0.001
start_x = 0.00
start_y = 0.00
end_x = 0.0
end_y = 0.09
target_ang = 180
start_ang = 0.0
cp = 150

shape_factor__ = 3.6
path_length__ = 0.1
v__ = 0.5

path_length_min = 0.01
path_length_max = 0.5

print(" v,     pre,      fol,      beta_max,  end_time")
for v in np.arange(0.1, 1.6, 0.1):
    _pre, _fol, _beta_max, _end_time = simulate180_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor__, path_length_min, path_length_max, v)
    print('{:>5.2f}'.format(v), '{:>9.5f}'.format(_pre), '{:>9.5f}'.format(_fol), '{:>7.2f}'.format(_beta_max), '{:>10.3f}'.format(_end_time) )



@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 1.0, 0.1)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    simulate180(tread, delta_t, start_x, start_y, end_x, end_y, target_ang, start_ang, cp, shape_factor, path_length_min, path_length_max, v)



 v,     pre,      fol,      beta_max,  end_time
 0.10   0.02700   0.02685    0.09      1.767
 0.20   0.02700   0.02685    0.36      0.886
 0.30   0.02700   0.02741    0.80      0.591
 0.40   0.02700   0.02781    1.43      0.443
 0.50   0.02700   0.02867    2.23      0.354
 0.60   0.02700   0.03023    3.21      0.295
 0.70   0.02700   0.03187    4.38      0.252
 0.80   0.02700   0.03241    5.71      0.221
 0.90   0.02700   0.03440    7.24      0.196
 1.00   0.02700   0.03307    8.92      0.179
 1.10   0.02700   0.03265   10.79      0.165
 1.20   0.02700   0.03016   12.84      0.154
 1.30   0.02700   0.02696   15.08      0.145
 1.40   0.02700   0.02288   17.45      0.138
 1.50   0.02700   0.01975   19.98      0.132


interactive(children=(FloatSlider(value=3.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

## ハーフ 斜めS2D 45度ターン

In [10]:
from pylab import rcParams
rcParams['figure.figsize'] = 8,8

# 設定パラメータ
tread = 0.038
delta_t = 0.001
start_x = -0.09
start_y = 0.00
end_x = 0.0
end_y = 0.045
target_ang = 45
start_ang = 0.0
cp = 250

shape_factor__ = 4
path_length__ = 0.04
v__ = 1.5

print(" v,     pre,      fol,      beta_max,  end_time")
for v in np.arange(0.1, 1.6, 0.1):
    _pre, _fol, _beta_max, _end_time = simulate_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor__, path_length__, v)
    print('{:>5.2f}'.format(v), '{:>9.5f}'.format(_pre), '{:>9.5f}'.format(_fol), '{:>7.2f}'.format(_beta_max), '{:>10.3f}'.format(_end_time) )


@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 2.0, 0.1)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    simulate(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length, v)


 v,     pre,      fol,      beta_max,  end_time
 0.10   0.02513   0.04356    0.06      0.391
 0.20   0.02506   0.04343    0.24      0.196
 0.30   0.02511   0.04368    0.53      0.130
 0.40   0.02487   0.04361    0.94      0.098
 0.50   0.02485   0.04403    1.47      0.078
 0.60   0.02472   0.04433    2.12      0.065
 0.70   0.02390   0.04444    2.88      0.056
 0.80   0.02354   0.04407    3.76      0.050
 0.90   0.02308   0.04318    4.74      0.046
 1.00   0.02251   0.04119    5.82      0.044
 1.10   0.02186   0.03967    6.97      0.042
 1.20   0.02110   0.03743    8.19      0.041
 1.30   0.02025   0.03547    9.46      0.040
 1.40   0.01931   0.03379   10.76      0.039
 1.50   0.01824   0.03090   12.07      0.039


interactive(children=(FloatSlider(value=4.0, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

## ハーフ 斜めS2D 135度ターン

In [11]:
from pylab import rcParams
rcParams['figure.figsize'] = 8,8

# 設定パラメータ
tread = 0.038
delta_t = 0.001
start_x = -0.09
start_y = 0.00
end_x = -0.045
end_y = 0.09
target_ang = 135
start_ang = 0.0
cp = 100

shape_factor__ = 4
path_length__ = 0.10
v__ = 0.5

print(" v,     pre,      fol,      beta_max,  end_time")
for v in np.arange(0.1, 1.6, 0.1):
    _pre, _fol, _beta_max, _end_time = simulate_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor__, path_length__, v)
    print('{:>5.2f}'.format(v), '{:>9.5f}'.format(_pre), '{:>9.5f}'.format(_fol), '{:>7.2f}'.format(_beta_max), '{:>10.3f}'.format(_end_time) )


@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 1.0, 0.1)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    simulate(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length, v)


 v,     pre,      fol,      beta_max,  end_time
 0.10   0.04540   0.03758    0.18      0.978
 0.20   0.04504   0.03752    0.71      0.490
 0.30   0.04475   0.03774    1.59      0.327
 0.40   0.04402   0.03830    2.83      0.245
 0.50   0.04313   0.03890    4.41      0.196
 0.60   0.04176   0.03857    6.33      0.165
 0.70   0.04053   0.03607    8.57      0.146
 0.80   0.03815   0.03253   11.09      0.133
 0.90   0.03608   0.02702   13.86      0.125
 1.00   0.03365   0.02100   16.82      0.119
 1.10   0.03084   0.01383   19.91      0.115
 1.20   0.02766   0.00489   23.06      0.113
 1.30   0.02413  -0.00515   26.21      0.112
 1.40   0.02029  -0.01516   29.32      0.111
 1.50   0.01488  -0.02826   32.35      0.112


interactive(children=(FloatSlider(value=4.0, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

## ハーフ 斜め 90度ターン

In [12]:
from pylab import rcParams
rcParams['figure.figsize'] = 8,8

# 設定パラメータ
tread = 0.038
delta_t = 0.001
start_x = -0.045
start_y = 0.00
end_x = -0.045
end_y = 0.09
target_ang = 90
start_ang = 45
cp = 100

shape_factor__ = 4
path_length__ = 0.05
v__ = 0.5

print(" v,     pre,      fol,      beta_max,  end_time")
for v in np.arange(0.1, 1.6, 0.1):
    _pre, _fol, _beta_max, _end_time = simulate_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor__, path_length__, v)
    print('{:>5.2f}'.format(v), '{:>9.5f}'.format(_pre), '{:>9.5f}'.format(_fol), '{:>7.2f}'.format(_beta_max), '{:>10.3f}'.format(_end_time) )


@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 1.0, 0.1)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    simulate(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length, v)


 v,     pre,      fol,      beta_max,  end_time
 0.10   0.03433   0.03423    0.24      0.489
 0.20   0.03407   0.03428    0.94      0.245
 0.30   0.03361   0.03471    2.12      0.163
 0.40   0.03332   0.03532    3.76      0.122
 0.50   0.03231   0.03427    5.86      0.101
 0.60   0.03145   0.03216    8.38      0.089
 0.70   0.03032   0.02907   11.25      0.082
 0.80   0.02823   0.02498   14.39      0.078
 0.90   0.02656   0.02000   17.69      0.076
 1.00   0.02465   0.01348   21.04      0.076
 1.10   0.02254   0.00699   24.36      0.076
 1.20   0.02020  -0.00066   27.59      0.077
 1.30   0.01762  -0.00977   30.69      0.079
 1.40   0.01483  -0.01924   33.61      0.081
 1.50   0.01181  -0.02906   36.38      0.083


interactive(children=(FloatSlider(value=4.0, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

# ハーフ斜め D2S 45度ターン

In [13]:
from pylab import rcParams
rcParams['figure.figsize'] = 8,8

# 設定パラメータ
tread = 0.038
delta_t = 0.001
start_x = -0.045
start_y = 0.0
end_x = 0.0
end_y = 0.09
target_ang = 45
start_ang = 45
cp = 100

shape_factor__ = 4
path_length__ = 0.04
v__ = 0.5

print(" v,     pre,      fol,      beta_max,  end_time")
for v in np.arange(0.1, 1.6, 0.1):
    _pre, _fol, _beta_max, _end_time = simulate_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor__, path_length__, v)
    print('{:>5.2f}'.format(v), '{:>9.5f}'.format(_pre), '{:>9.5f}'.format(_fol), '{:>7.2f}'.format(_beta_max), '{:>10.3f}'.format(_end_time) )


@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 1.0, 0.1)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    simulate(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length, v)


 v,     pre,      fol,      beta_max,  end_time
 0.10   0.04371   0.02498    0.15      0.391
 0.20   0.04346   0.02502    0.59      0.196
 0.30   0.04320   0.02557    1.33      0.130
 0.40   0.04256   0.02590    2.36      0.098
 0.50   0.04200   0.02582    3.67      0.080
 0.60   0.04121   0.02414    5.25      0.071
 0.70   0.03960   0.02156    7.05      0.066
 0.80   0.03832   0.01859    9.01      0.063
 0.90   0.03683   0.01460   11.06      0.062
 1.00   0.03513   0.00999   13.13      0.062
 1.10   0.03320   0.00445   15.16      0.063
 1.20   0.03102  -0.00229   17.13      0.065
 1.30   0.02866  -0.00798   18.98      0.066
 1.40   0.02599  -0.01642   20.75      0.069
 1.50   0.02312  -0.02379   22.36      0.071


interactive(children=(FloatSlider(value=4.0, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

# ハーフ斜め D2S 135度ターン

In [14]:
from pylab import rcParams
rcParams['figure.figsize'] = 8,8

# 設定パラメータ
tread = 0.038
delta_t = 0.001
start_x = -0.045
start_y = 0.00
end_x = -0.09
end_y = 0.09
target_ang = 135
start_ang = 45
cp = 100

shape_factor__ = 4
path_length__ = 0.10
v__ = 0.5

print(" v,     pre,      fol,      beta_max,  end_time")
for v in np.arange(0.1, 1.6, 0.1):
    _pre, _fol, _beta_max, _end_time = simulate_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor__, path_length__, v)
    print('{:>5.2f}'.format(v), '{:>9.5f}'.format(_pre), '{:>9.5f}'.format(_fol), '{:>7.2f}'.format(_beta_max), '{:>10.3f}'.format(_end_time) )



@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 1.0, 0.1)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    simulate(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length, v)


 v,     pre,      fol,      beta_max,  end_time
 0.10   0.03767   0.04530    0.18      0.978
 0.20   0.03732   0.04524    0.71      0.490
 0.30   0.03703   0.04546    1.59      0.327
 0.40   0.03630   0.04602    2.83      0.245
 0.50   0.03541   0.04662    4.41      0.196
 0.60   0.03404   0.04629    6.33      0.165
 0.70   0.03281   0.04379    8.57      0.146
 0.80   0.03043   0.04025   11.09      0.133
 0.90   0.02836   0.03474   13.86      0.125
 1.00   0.02593   0.02872   16.82      0.119
 1.10   0.02312   0.02155   19.91      0.115
 1.20   0.01994   0.01261   23.06      0.113
 1.30   0.01641   0.00257   26.21      0.112
 1.40   0.01257  -0.00744   29.32      0.111
 1.50   0.00716  -0.02053   32.35      0.112


interactive(children=(FloatSlider(value=4.0, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…